[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/morganmcg1/deep-research-bot/blob/main/notebooks/01_simple_agent.ipynb)


# Introduction to Building LLM Agents with Tools and Tracing

<!--- @wandbcode{fc-london-workshop-2025} -->

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git
```


In [1]:
!uv pip install -qqq git+https://github.com/morganmcg1/deep-research-bot.git

In [2]:
# Global Configuration & Setup
import os
import inspect
import json
import weave # Must import weave before litellm for auto-patching
import openai
from enum import Enum
from pydantic import BaseModel, Field
from rich.panel import Panel
from rich.markdown import Markdown
from rich.console import Console as RichConsole
from exa_py import Exa
from typing import Any, Callable, get_type_hints

### Add your API Keys
We'll need a wandb api key and an exa api key

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [32]:
# or

os.environ["WANDB_API_KEY"] = "8268ecbd4ece505abd6d0f3da69bc66c9502480f"
os.environ["EXA_API_KEY"] = "a128cb84-30a1-4741-bf1b-b5e6ded37519"

# or colab secrets:

# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
# os.environ["EXA_API_KEY"] = userdata.get("EXA_API_KEY")

## Model Settings
Define a model to use, as we are going to use tool calling you need a capable model like `Kimi-K2` or `GLM`

In [33]:
class Console(RichConsole):
    def md(self, text):
        return self.print(Markdown(text))

console = Console()

MODEL_SMALL = "Qwen/Qwen3-235B-A22B-Instruct-2507"
MODEL_MEDIUM = "zai-org/GLM-4.5"
MODEL_LARGE = "moonshotai/Kimi-K2-Instruct"

WANDB_ENTITY = "team-jup"
WANDB_PROJECT = "london-workshop-2025"

oai_client = openai.OpenAI(
    base_url='https://api.inference.wandb.ai/v1',
    api_key=os.getenv("WANDB_API_KEY"),
    project=f"{WANDB_ENTITY}/{WANDB_PROJECT}")
exa_client = Exa(api_key=os.getenv("EXA_API_KEY"))

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [7]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

Output()

weave: wandb version 0.22.3 is available!  To upgrade, please run:
weave:  $ pip install wandb --upgrade
weave: Logged in as Weights & Biases user: julien-fauqueur.
weave: View Weave data at https://wandb.ai/team-jup/london-workshop-2025/weave


## 1. Basic LLM Call with OpenAI SDK

Let's start with a simple call to the LLM using/

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/01_trace.png?raw=1)

In [6]:
# Define a simple message list (conversation history)
messages = [{"role": "user", "content": "Hello, LLM! How does an AI agent work?"}]

# Make the call
response = resp = oai_client.chat.completions.create(
    model = MODEL_SMALL,
    messages=messages,
    max_tokens=400,
)
# Print the response content
assistant_response = response.choices[0].message.content
console.md(f"LLM Response:\\n{assistant_response}")

# Click on the 🍩 linke below to see the trace in Weave 👇

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4e78-a566-7edf-a22c-a7331766076f


LLM Response:\nHello! Great question! 😊                                                                           

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "assistant" that can operate autonomously or semi-autonomously.         

Here’s a breakdown of how an AI agent works:                                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   1. Perception                                                   

The agent gathers information from its environment using sensors or data inputs. This could be:                    

 • Text input (like your message to me)                                                                            
 • Camera or microphone data (in robots or voice assistants)                                                       
 • Sensor readings (in self-driving cars)                                                                          
 • Database queries or web scraping                                                                                

👉 Example: A chatbot reads your message; a self-driving car sees traffic signs via cameras.                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     2. Decision-Making (Reasoning & Planning)                                     

Based on what it perceives, the agent uses algorithms or models (like machine learning or rule-based systems) to   
decide what to do.                                                                                                 

This can involve:                                                                                                  

 • Pattern recognition (e.g., understanding language)                                                              
 • Prediction (e.g., what you might ask next)                                                                      
 • Planning (e.g., breaking down a task into steps)                                                                
 • Learning from experience (in advanced agents)                                                                   

👉 Example: A virtual assistant decides whether to search the web, answer directly, or ask for clarification.      

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     3. Action                                                     

The agent performs an action using actuators or output mechanisms:                                                 

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Adjusting thermostat settings                                                                                   
 • Placing an online order                                                                                         

👉 Example: I generate and send you this response.                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                     4. Feedback Loop (Optional but Powerful)                                      

Many agents learn from outcomes:                                                                                   

 • Did the user like the answer? (via feedback or follow-up)                                                       
 • Did the robot reach its destination?   

Because we imported `weave` and called `weave.init()`, the OpenAI SDK call above was automatically traced. You can open your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [8]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: list[dict[str, Any]], **kwargs) -> str:
    "Call a model with the given messages and kwargs."
    response = oai_client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=400,
        **kwargs
    )

    return response.choices[0].message

response = call_model(model_name=MODEL_SMALL, messages=messages)
console.md(response.content)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4e7d-0b5b-7467-957a-0c61ce9b0f02


Hello! Great question! 😊                                                                                          

An AI agent is a software system that perceives its environment, makes decisions, and takes actions to achieve     
specific goals. Think of it like a digital "assistant" that can think and act autonomously, to some degree. Here's 
how it works, broken down into key components:                                                                     

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 🔍 1. Perception                                                  

The agent gathers information from its environment using:                                                          

 • Sensors (in robotics)                                                                                           
 • User inputs (like text or voice)                                                                                
 • Data from APIs, databases, or the web                                                                           

For example: A chatbot reads your message; a self-driving car uses cameras and radar.                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                   🧠 2. Decision-Making (Reasoning & Planning)                                    

This is where the "intelligence" comes in. The agent processes the input and decides what to do using:             

 • Rules or logic (e.g., if-then statements)                                                                       
 • Machine learning models (like neural networks)                                                                  
 • Search algorithms (e.g., finding the best move in a game)                                                       
 • Large Language Models (LLMs) — especially in modern agents for understanding and generating language            

Modern AI agents often use LLMs as their "brain" to reason, plan, and generate responses.                          

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                   🤖 3. Action                                                    

The agent performs actions based on its decisions:                                                                 

 • Sending a message (like this one!)                                                                              
 • Moving a robot arm                                                                                              
 • Clicking a button in software                                                                                   
 • Calling an API (e.g., booking a flight, checking weather)                                                       

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                    🔄 4. Feedback Loop (Learning & Adaptation)                                    

Some agents learn from experience:                                                                                 

 • Reinforcement learning: learns by trial and error                                                               
 • Updating models based on user feedback                                                                          
 • Remembering past interactions (with memory systems)                                                             

This helps them improve over time.                                                                                 

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                      🧩 Exam

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/02_nested_trace.png?raw=1)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://platform.openai.com/docs/guides/function-calling

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/function-calling-diagram-steps.png?raw=1)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [9]:
@weave.op
def add_numbers(a: int, b: int) -> int:
    """Use this tool to add numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [10]:
add_numbers(1, 2)

weave: retry_attempt


3

weave: retry_attempt


In [11]:
# this doesn't work...
call_model(model_name=MODEL_SMALL, messages=messages, tools=[add_numbers])

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4e8c-6edf-706f-8f9c-a23f118da5ab


TypeError: Object of type function is not JSON serializable

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [12]:
# Manually define the tool schema
tool_add_numbers_schema = {
    "type": "function",
    "function": {
        "name": "add_numbers",
        "description": "Adds two numbers.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "integer",
                    "description": "The first number."
                },
                "b": {
                    "type": "integer",
                    "description": "The second number."
                }
            },
            "required": ["a", "b"]
        }
    }
}

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [18]:
messages = [
    {"role": "system", "content": "You are a helpful assistant use tools to answer questions."},
    {"role": "user", "content": "My lucky numbers are 77 and 11. What is their sum? "}]
response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[tool_add_numbers_schema])
console.print(response)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4eaa-3be2-73e4-a311-7ff921c5bdec


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-dfba94b007b244048bdcda1a9a2e8ed4',
            function=Function(arguments='{"a": 77, "b": 11}', name='add_numbers'),
            type='function'
        )
    ],
    reasoning_content=None
)

## Manual Tool Call
The LLM's response might contain a request to call our tool (`response.choices[0].message.tool_calls`) or it might respond directly (`response.choices[0].message.content`). If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [19]:
if response.tool_calls:
    console.print("LLM requested a tool call:")
    for tool_call in response.tool_calls:
        function_name = tool_call.function.name
        function_args_str = tool_call.function.arguments
        function_args = json.loads(function_args_str)
        console.print(f"  - Tool: {function_name}, Args: {function_args_str}")
        if function_name == "add_numbers":
            tool_result_content = add_numbers(**function_args)

console.print(f"Final Result: {tool_result_content}")

LLM requested a tool call:

- Tool: add_numbers, Args: {"a": 77, "b": 11}

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec0-d4cc-7886-a884-9ed189e5140a


Final Result: 88

We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [20]:
messages.append(response.model_dump())

In [21]:
messages.append({
    "tool_call_id": tool_call.id,
    "role": "tool",
    "name": function_name,
    "content": str(tool_result_content)
})

In [22]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant use tools to answer questions.'},
 {'role': 'user',
  'content': 'My lucky numbers are 77 and 11. What is their sum? '},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'annotations': None,
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'chatcmpl-tool-dfba94b007b244048bdcda1a9a2e8ed4',
    'function': {'arguments': '{"a": 77, "b": 11}', 'name': 'add_numbers'},
    'type': 'function'}],
  'reasoning_content': None},
 {'tool_call_id': 'chatcmpl-tool-dfba94b007b244048bdcda1a9a2e8ed4',
  'role': 'tool',
  'name': 'add_numbers',
  'content': '88'}]

You should have a sequence of messages like this:


In [23]:
[m["role"] for m in messages]

['system', 'user', 'assistant', 'tool']

Now call the model again with the new messages and it will use the tool call result to answer the question

In [24]:
final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.print(final_response.content)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec1-60e7-767e-b194-f5c1b98550eb


The sum of your lucky numbers, 77 and 11, is **88**.

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [25]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a tool-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    schema = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": inspect.getdoc(func).split("\\n")[0] if inspect.getdoc(func) else "",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    }

    docstring = inspect.getdoc(func)
    param_descriptions = {}
    if docstring:
        args_section = False
        current_param = None
        for line in docstring.split('\\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped:
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" "): # Heuristic: end of args section
                     args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle list[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by list or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: OpenAI schema doesn't officially use default, but useful metadata

        schema["function"]["parameters"]["properties"][name] = param_schema
        if is_required:
            schema["function"]["parameters"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [26]:
tool_schema = generate_tool_schema(add_numbers)
console.print(tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [27]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        console.print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [28]:
add_numbers = function_tool(add_numbers)
console.print(add_numbers.tool_schema)

{
    'type': 'function',
    'function': {
        'name': 'add_numbers',
        'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
        'parameters': {
            'type': 'object',
            'properties': {
                'a': {'type': 'integer', 'description': ''},
                'b': {'type': 'integer', 'description': ''}
            },
            'required': ['a', 'b']
        }
    }
}

In [29]:
add_numbers.tool_schema

{'type': 'function',
 'function': {'name': 'add_numbers',
  'description': 'Use this tool to add numbers.\nArgs:\n    a: The first number.\n    b: The second number.',
  'parameters': {'type': 'object',
   'properties': {'a': {'type': 'integer', 'description': ''},
    'b': {'type': 'integer', 'description': ''}},
   'required': ['a', 'b']}}}

and call the tool =)

In [30]:
add_numbers(1, 2)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec3-2857-7322-b775-6f4908a325f8


3

### 3.1 Real Example using an API based tool

We are going to use the [EXA search API](https://docs.exa.ai/reference/getting-started).
- How does [EXA search works](https://docs.exa.ai/reference/how-exa-search-works#how-exa-search-works)
- Using exa search [as tool calling](https://docs.exa.ai/reference/openai-tool-calling)

In [34]:
query = "Recipes for cooking seabass?"

search_res = exa_client.search_and_contents(query=query, type='auto', num_results=4)

In [35]:
console.print(search_res)

SearchResponse(
    results=[
        Result(
            url='https://www.bbc.co.uk/food/sea_bass',
            id='https://www.bbc.co.uk/food/sea_bass',
            title='Sea bass recipes',
            score=None,
            published_date='2025-03-21T00:00:00.000Z',
            author='',
            image='',
            favicon=None,
            subpages=None,
            extras=None,
            text="[Close menu](https://www.bbc.co.uk/food/sea_bass)\n\n[Food](https://www.bbc.co.uk/food)\n\n# Sea 
bass recipes\n\nA superb sweet, white, textured fish, sea bass is a popular feature of menus but you can easily 
cook it at home. Our sea bass recipes make quick work or a whole fish or fillets.Try Nadiya's easy sea bass bake 
with potatoes, or add tarragon flavoured breadcrumbs as in Mary Berry's baked sea bass fillets.\n\n[Baked sea bass 
on potatoes](https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811)\n\nby José Pizarro\n\nWhole sea bass
is roasted on a layer of potatoes with herbs and a punchy dressing, served with a flavour-packed pickled walnut 
salad. This is a really easy way to make a simple fish dish a spectacular centrepiece.\n\nMain course\n\n## More 
sea bass recipes\n\n[Featured](https://www.bbc.co.uk/food/sea_bass#featured-content) 
[All](https://www.bbc.co.uk/food/sea_bass/a-z#featured-content)\n\n[**Pan-fried sea bass in harissa butter** by 
Georgina Hayden\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540)\n\n[**Tarragon 
crusted sea bass** by Mary Berry\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411)\n\n[**Sea bass with tomato, olive and herb 
sauce** by Mitch Tonks\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150)\n\n[**Sea
bass with herb crust, pepper and balsamic dressing** by Bryn Williams\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sea_bass_herb_crust_13119)\n\n[**Cured sea bass with purple sprouting 
broccoli, hazelnut pesto and bagna cauda** by Adam Byatt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/cured_sea_bass_with_28896)\n\n[**Whole roasted sea bass with romesco 
sauce and potatoes** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/roasted_sea_bass_romesco_06487)\n\n[**Whole baked sea bass** by Angela 
Hartnett\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/whole_baked_sea_bass_88065)\n\n[**Baked sea bass on
potatoes** by José Pizarro\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811)\n\n[**Mexican fish with refried beans** 
by Rick Stein\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/butterfield_grilled_bass_01988)\n\n[**Baked 
sea bass with fennel and porcini mushroom** by Theo Randall\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/baked_seabass_with_96694)\n\n[**Bouillabaisse** by Mary 
Berry\\\n\\\nMain course](https://www.bbc.co.uk/food/recipes/bouillabaisse_00342)\n\n[**Harissa baked sea bass with
baby potatoes, mint salsa and Mediterranean salad** by Donal Skehan\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/harissa_baked_sea_bass_91056)\n\n[**Sicilian-style fish and couscous** 
by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sicilian-style_fish_and_07938)\n\n[**Pan-roasted sea bass with basil 
butter sauce** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/wild_sea_bass_with_basil_25408)\n\n[**Sweet and sour fish with Sicilian 
orange salad** by Matt Tebbutt\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/sweet_and_sour_fish_with_73513)\n\n[**Yunnan-style grilled sea bass with
twice-cooked pork** by Ching-He Huang\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/grilled_sea_bass_with_96195)\n\n[**Salt-baked sea bass with beurre 
blanc** by Richard Bainbridge\\\n\\\nMain 
course](https://www.bbc.co.uk/food/recipes/salt-baked_bream_with_78937)\n\n[**Roasted sea bass with wild 
mushrooms** by Matt Teb

Let's explore the payload

In [36]:
console.md("\n-------------------\n".join(result.text for result in search_res.results))

]8;id=675272;https://www.bbc.co.uk/food/sea_bass\Close menu]8;;\                                                                                                         

]8;id=618634;https://www.bbc.co.uk/food\Food]8;;\                                                                                                               

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃                                                Sea bass recipes                                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

A superb sweet, white, textured fish, sea bass is a popular feature of menus but you can easily cook it at home.   
Our sea bass recipes make quick work or a whole fish or fillets.Try Nadiya's easy sea bass bake with potatoes, or  
add tarragon flavoured breadcrumbs as in Mary Berry's baked sea bass fillets.                                      

]8;id=449798;https://www.bbc.co.uk/food/recipes/whole_sea_bass_baked_on_19811\Baked sea bass on potatoes]8;;\                                                                                         

by José Pizarro                                                                                                    

Whole sea bass is roasted on a layer of potatoes with herbs and a punchy dressing, served with a flavour-packed    
pickled walnut salad. This is a really easy way to make a simple fish dish a spectacular centrepiece.              

Main course                                                                                                        


                                               More sea bass recipes                                               

]8;id=329842;https://www.bbc.co.uk/food/sea_bass#featured-content\Featured]8;;\ ]8;id=440972;https://www.bbc.co.uk/food/sea_bass/a-z#featured-content\All]8;;\                                                                                                       

]8;id=721630;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\Pan-fried sea bass in harissa butter]8;;\]8;id=332610;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\ by Georgina Hayden]8;;\                                                            
                                                                                                                   
]8;id=332610;https://www.bbc.co.uk/food/recipes/pan-fried_sea_bass_in_88540\Main course]8;;\                                                                                                        

]8;id=942660;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\Tarragon crusted sea bass]8;;\]8;id=938662;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\ by Mary Berry]8;;\                                                                            
                                                                                                                   
]8;id=938662;https://www.bbc.co.uk/food/recipes/tarragon_crusted_sea_93411\Main course]8;;\                                                                                                        

]8;id=30359;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\Sea bass with tomato, olive and herb sauce]8;;\]8;id=42330;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\ by Mitch Tonks]8;;\                                                          
                                                                                                                   
]8;id=42330;https://www.bbc.co.uk/food/recipes/seabasswithtomatooli_72150\Main course]8;;\                                                                                                        

]8;id=533552;https://www.bbc.co.uk/food/recipes/sea_bass_herb_crust_13119\Sea bass with herb cru

In [37]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    output = []
    for result in search_results.results:
        output.append(
            {"title": result.title,
            "text": result.text,
            "url": result.url
            }
        )
    return output



In [39]:
exa_search.tool_schema

{'type': 'function',
 'function': {'name': 'exa_search',
  'description': 'Perform a search query on the web and retrieve the most relevant URLs and web content.\n\nThis function uses the Exa search API to find relevant web pages based on the query\nand returns their titles, text content, and URLs.\n\nArgs:\n    query: The search query. Use detailed, specific queries for better results.\n           The quality of results depends on the specificity of the query.\n    num_results: The number of search results to retrieve. Defaults to 5.\n\nReturns:\n    A list of dictionaries, each containing:\n        - title: The title of the web page\n        - text: The text content of the web page\n        - url: The URL of the web page',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string', 'description': ''},
    'num_results': {'type': 'integer', 'description': '', 'default': 5}},
   'required': ['query']}}}

We get a list of results with the relevant metadata.

In [40]:
search_results = exa_search("How do I cook seabass?")
console.print(search_results)

weave: retry_attempt
weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec8-0e45-70fa-81e7-1112b35c897c


[
    {
        'title': "Chilean Sea Bass Recipe {Restaurant Style} - The Big Man's World ®",
        'text': '[Skip to content](https://thebigmansworld.com/thebigmansworld.com#main-content)\n\n# Chilean Sea 
Bass Recipe\n\nBy [Arman Liew](https://thebigmansworld.com/arman-liew/)published on May 20, 2025\n\n[Jump to Recipe
▼](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)\n\nReader Rating\n\nTotal Time
10 minutesmins\n\nServings 4servings\n\n_This post may contain affiliate links. See my [disclosure 
policy](https://thebigmansworld.com/privacy-policy/)._\n\n_This pan seared **Chilean sea bass recipe** comes 
together in under 10 minutes and cooks perfectly every single time! It’s easy, elegant, and delicious._\n\n**★★★★★ 
REVIEW**\n\n_“OMG, this recipe was the easiest and the most delicious! My sister had never had sea bass before, so 
I made it for both of us. I did get it without the skin, and it was absolutely amazing! If you haven’t tried this, 
please use this recipe, you won’t regret it, the absolute best!”_ – Pam\n\nTable of Contents\n\n1. [Key 
Ingredients](https://thebigmansworld.com/thebigmansworld.com#h-key-ingredients)\n2. [How to cook Chilean sea 
bass](https://thebigmansworld.com/thebigmansworld.com#h-how-to-cook-chilean-sea-bass)\n3. [Chilean sea bass cooking
tips](https://thebigmansworld.com/thebigmansworld.com#h-chilean-sea-bass-cooking-tips)\n4. [How to 
serve](https://thebigmansworld.com/thebigmansworld.com#h-how-to-serve)\n5. [Seared Chilean Sea Bass Recipe (Recipe 
Card)](https://thebigmansworld.com/thebigmansworld.com#wprm-recipe-container-61837)\n\nFriends, if you are looking 
for a simple fish dinner that’s elegant enough for date night, a dinner party, or a special occasion, my quick, 
pan-seared Chilean sea bass recipe is a must-try. While it’s a popular choice at fine dining restaurants or elegant
events, making it at home is SO easy and tastes just as good- if not better!\n\nIt’s so tender, flaky, and 
naturally buttery that the sea bass practically melts in your mouth. I love that it takes just ten minutes to make 
(prep AND cook time included) and has minimal clean-up. It’s drizzled with a simple garlic lemon butter, which lets
the natural flavor of the Chilean sea bass shine.\n\nIf you love elegant seafood dinners, try my **[parmesan 
crusted tilapia](https://thebigmansworld.com/parmesan-crusted-tilapia/)**, **[grilled tuna 
steak](https://thebigmansworld.com/grilled-tuna-steak/)**, **[roasted 
branzino](https://thebigmansworld.com/branzino-recipe/)**, and 
**[rockfish](https://thebigmansworld.com/pan-seared-rockfish-recipe/)** next.\n\nWhat is Chilean Sea 
Bass?\n\nChilean sea bass (also called Patagonian toothfish, mero, or icefish), is a deep-water species native to 
the cold sub-Antarctic regions of the Pacific, southern Atlantic, and Indian Oceans.\n\nIt’s high in protein and 
super low in carbs. It’s famous for its silky, buttery texture and mild flavor (no fishy taste). It’s considered a 
premium fish due as its hard to overcook and super versatile.\n\n## Key Ingredients\n\n- **Chilean sea bass 
fillets**. Either fresh or frozen fillets.\xa0Because of the slightly higher price tag, I prefer to get the fillets
from a fishmonger over a grocery store. When choosing sea bass, opt for fillets that have no strong odors, no dry 
spots, and are firm to touch.\n- **Kosher salt and black pepper**. To taste.\n- **Unsalted butter**. Pan-frying 
white fish fillets in butter takes the dish’s flavor to another level and cooks the fish perfectly. Olive oil will 
also work.\n- **Garlic**. Freshly minced garlic, not the jarred kind!\n- **Lemon**. Fresh lemons to squeeze over 
the freshly cooked fish.\n\n## How to cook Chilean sea bass\n\n_This is an overview with step-by-step photos. Full 
ingredients & instructions are in the [recipe card 
below](https://thebigmansworld.com/chilean-sea-bass-recipe/#wprm-recipe-container-61837)._\n\n**Step 1-** Pat dry 
the fish filets wi

In [41]:
messages = [
    {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
    {"role": "user", "content": "How do I cook seabass?"}]

response = call_model(model_name=MODEL_SMALL, messages=messages, tools=[exa_search.tool_schema])
console.print(response)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec8-95c5-7138-9263-8f5700ef07f7


ChatCompletionMessage(
    content=None,
    refusal=None,
    role='assistant',
    annotations=None,
    audio=None,
    function_call=None,
    tool_calls=[
        ChatCompletionMessageFunctionToolCall(
            id='chatcmpl-tool-2c9d50d94e754a62af20dbc389a3c809',
            function=Function(
                arguments='{"query": "how to cook seabass step by step", "num_results": 5}',
                name='exa_search'
            ),
            type='function'
        )
    ],
    reasoning_content=None
)

Let's create some helper functions to perform the tool calls

In [43]:
def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], tool_calls: list[Callable]) -> list[dict]:
    "Perform the tool calls and return the messages with the tool call results"
    messages = []
    if not tool_calls:
        return messages
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        with console.status(f"[bold cyan]Executing {function_name}...[/bold cyan]"):
            tool = get_tool(tools, function_name)
            tool_response = tool(**function_args) # doesn't handle async

        # Create panel content
        panel_content = f"[bold cyan]🔧 Tool Call:[/bold cyan] {function_name}\n\n"
        panel_content += f"[dim]Args: {tool_call.function.arguments}[/dim]\n\n"

        if isinstance(tool_response, list):
            panel_content += f"[green]✓[/green] Found {len(tool_response)} results"
        else:
            panel_content += f"[green]✓[/green] {function_name} executed successfully"

        console.print(Panel(panel_content, border_style="cyan"))

        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "content": str(tool_response),
        })
    return messages

In [44]:

# add the tool call result to the messages
messages.append(response.model_dump())
messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))
messages.append({
    "role": "user",
    "content": "Answer my previous query based on the search results.",})

final_response = call_model(model_name=MODEL_SMALL, messages=messages)
console.rule("Final Model Response")
console.md(final_response.content)


weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec9-2b16-7d98-96b2-6c2d5aa26c15


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "how to cook seabass step by step", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ec9-2f38-7c97-b6a1-363dd899f33b


────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Cooking seabass is simple and can be done in multiple ways, including pan-searing, baking, grilling, or using an   
air fryer. Here’s a concise guide based on the search results:                                                     

                                          Popular Methods to Cook Seabass                                          

                                         1. Pan-Seared Seabass (Stovetop)                                          

This method gives a crispy exterior and tender, flaky interior.                                                    

Steps:                                                                                                             

 1 Prep the fish: Pat the fillets dry with paper towels to remove moisture. Season both sides with salt and pepper.
 2 Heat the pan: Use a non-stick or cast-iron skillet. Heat over medium-high heat with olive oil, butter, or a mix.
 3 Sear skin-side down: Place the fillets skin-side down and cook for 3–5 minutes until golden and crispy. Avoid   
   moving the fish until it releases easily.                                                                       
 4 Flip and finish: Turn and cook the other side for 2–4 minutes until the internal temperature reaches 140–145°F  
   (60–63°C) and the flesh flakes easily with a fork.                                                              
 5 Add flavor: Finish with a squeeze of lemon juice or a sauce like garlic butter, lemon-caper butter, or a        
   Mediterranean vegetable medley.                                                                                 

▌ Tip: For thicker fillets, transfer the skillet to a 425°F oven for 5 minutes after searing to ensure even      
▌ cooking.                                                                                                       

                                                 2. Baked Seabass                                                  

Ideal for a hands-off, moist result.                                                                               

Steps:                                                                                                             

 1 Preheat oven to 400°F (200°C).                                                                                  
 2 Season and prep: Brush fillets with melted butter or olive oil. Season with salt, pepper, paprika (optional),   
   and lemon slices on top.                                                                                        
 3 Bake: Place on a parchment-lined baking sheet. Bake for 12–18 minutes, depending

Let's wrap this in a function:

In [45]:
@weave.op
def research(query: str) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Return in markdown format."},
        {"role": "user", "content": query}]

    # call model with tools
    response = call_model(
        model_name=MODEL_SMALL,
        messages=messages,
        tools=[exa_search.tool_schema])

    # add the response to the messages
    messages.append(response.model_dump())

    # perform the tool calls
    messages.extend(perform_tool_calls(tools=[exa_search], tool_calls=response.tool_calls))

    # prompt the model to be grounded
    messages.append({"role": "user","content": "Answer my previous query based on the search results.",})

    final_response = call_model(model_name=MODEL_SMALL, messages=messages)
    return final_response.content

In [46]:
result = research("What are the most popular pokemons?")
console.md(result)

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ecb-4461-7fde-9284-71e21b6ee50c


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "most popular Pokémon of all time", "num_results": 5}                                           │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Based on the search results, the most popular Pokémon, according to fan surveys and online data, are:              

 1 Charizard – Frequently ranked as the most popular Pokémon, Charizard topped a survey of over 52,000 Reddit users
   with 1,107 votes. It is beloved for its powerful design, role in the anime as Ash’s companion, and iconic status
   in trading cards, including a rare first-edition card that sold for $220,000.                                   
 2 Gengar – Ranked second in popularity with 1,056 votes in the same Reddit survey, Gengar has grown from a        
   mysterious Ghost-type into a fan favorite due to its eerie yet charming design and cultural appeal, especially  
   during Halloween events.                                                                                        
 3 Arcanine – With 923 votes, Arcanine is praised for its majestic appearance and legendary speed, often described 
   as capable of running over 6,200 miles in a day.                                                                
 4 Bulbasaur – The most popular starter Pokémon, Bulbasaur received 710 votes and is cherished for its balanced    
   design, role in the original games and anime, and enduring appeal across generations.                           
 5 Eevee – Ranked ninth in the Reddit survey but highly popular overall, Eevee is adored for its cuteness and      
   versatility, with multiple evolution paths. It is especially favored among female fans.                         

Other notable mentions include:                                                                                    

 • Lucario – Ranked 7th in the Reddit poll and ranked #1 in a 2020 fan vote for the Sinnoh region, Lucario is      
   popular for its anime appearances and unique aura-based abilities.                                              
 • Pikachu – While the official mascot of Pokémon and globally iconic, Pikachu ranked only 44th in the Reddit      
   survey, indicating it's more recognized than preferred as a favorite.                                           
 • Greninja – Won the 20

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/04_pokedex.png?raw=1)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/05_agent.png?raw=1)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [47]:
@weave.op
def research_loop(query: str, max_turns: int = 4, tools = [exa_search, add_numbers]) -> str:
    messages = [
        {"role": "system", "content": "You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format."},
        {"role": "user", "content": query}]

    for turn in range(max_turns):
        console.rule(f"Agent Loop Turn {turn + 1}/{max_turns}")

        # call model with tools
        response = call_model(
            model_name=MODEL_MEDIUM,
            messages=messages,
            tools=[t.tool_schema for t in tools])

        # add the response to the messages
        messages.append(response.model_dump())

        # if the LLM requested tool calls, perform them
        if response.tool_calls:
            # perform the tool calls
            tool_outputs = perform_tool_calls(tools=tools, tool_calls=response.tool_calls)
            messages.extend(tool_outputs)
        # LLM gave content response
        elif response.content:
            console.rule("Final Model Response")
            console.md(response.content)
            return response.content
        else:
            print("LLM response had neither content nor tool calls. Stopping loop.")
            break

In [48]:
res = research_loop("What is the sum of the populations of the 2 major EU cities?")

─────────────────────────────────────────────── Agent Loop Turn 1/4 ───────────────────────────────────────────────

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4ed0-228f-7d5b-af1b-f6218541cec0


Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "largest cities European Union by population 2024 Paris Berlin", "num_results": 5}              │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/4 ───────────────────────────────────────────────

────────────────────────────────────────────── Final Model Response ───────────────────────────────────────────────

Now I have the search results. Looking at the data, I need to identify the 2 major EU cities by population and get 
their populations, then calculate the sum.                                                                         

From the search results, I can see several sources listing European cities by population. However, I need to be    
careful about which cities are actually in the EU (European Union). Some sources include cities from Russia, which 
is not in the EU, and the UK, which has left the EU (Brexit).                                                      

Looking at the results:                                                                                            

 1 From the first source (worldpopulationreview.com) - shows Paris at 11,346,800 and London at 9,840,740, but      
   London is no longer in the EU.                                                                                  
 2 From the Eurostat source - "In 2023, there were 5 metropolitan regions in the EU with over 5 million people. The
   EU's largest metropolitan region in 2023 was Paris in France, with a population of 12.4 million people, followed
   by the Spanish metropolitan regions of Madrid and Barcelona with 6.9 and 5.8 million inhabitants, respectively. 
   Berlin, with a total population of 5.5 million, and the Ruhrgebiet metropolitan region in Germany (5.1 million  
   residents), completed the top 5."                                                                               

This Eurostat source seems more authoritative for EU-specific data since it's from the European Commission's       
statistical office. According to this:                                                                             

 • Paris: 12.4 million                                                                                             
 • Madrid: 6.9 million                                                                                             
 • Barcelona: 5.8 million                                                                                          
 • Berlin: 5.5 million                                                                                             
 • Ruhrgebiet: 5.1 million                                                                                         

So the top 2 major EU cities would be Paris (12.4 million) and Madrid (6.9 million).                               

 3 From Wikipedia - "Berlin, the most populous city in the European Union" with 3,897,145 within city boundaries,  
   but this is city proper population, not metropolitan area

# 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [49]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: list[dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

In [50]:
class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: list[Callable], state_class: BaseModel = AgentState):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
        self.state_class = state_class

    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        try:
            # call model with tools
            response = call_model(
                model_name=self.model_name,
                messages=messages,
                tools=[t.tool_schema for t in self.tools])

            # add the response to the messages
            messages.append(response.model_dump())

            # if the LLM requested tool calls, perform them
            if response.tool_calls:
                # perform the tool calls
                tool_outputs = perform_tool_calls(tools=self.tools, tool_calls=response.tool_calls)
                messages.extend(tool_outputs)

            # LLM gave content response
            else:
                final_assistant_content = response.content
        except Exception as e:
            console.print(f"ERROR in Agent Step: {e}")
            # Add an error message to history to indicate failure
            messages.append({"role": "assistant", "content": f"Agent error in step: {str(e)}"})
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
        return self.state_class(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10, **state_kwargs: Any) -> AgentState:
        state = self.state_class(messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": user_prompt}], **state_kwargs)
        for _ in range(max_turns):
            console.rule(f"Agent Loop Turn {state.step+1}/{max_turns}")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state



![](https://github.com/morganmcg1/deep-research-bot/blob/main/notebooks/images/07_simple_agent.png?raw=1)

In [51]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search, add_numbers],
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

weave: retry_attempt
weave: retry_attempt


Output()

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4eda-84f6-7644-981e-7cef8fb13bf2


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "Ash's first three Pokémon weights", "num_results": 5}                                          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

Output()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: exa_search                                                                                        │
│                                                                                                                 │
│ Args: {"query": "weight of Ash's first three Pokémon Bulbasaur Charmander Squirtle", "num_results": 5}          │
│                                                                                                                 │
│ ✓ Found 5 results                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 3/3 ───────────────────────────────────────────────

Final response: Based on the search results, Ash's first three Pokémon are **Bulbasaur**, **Charmander**, and **Squirtle**. Their individual weights are:

- **Bulbasaur**: 15.2 lbs (6.9 kg)  
- **Charmander**: 18.7 lbs (8.5 kg)  
- **Squirtle**: 19.8 lbs (9.0 kg)  

### Combined Weight:
**15.2 + 18.7 + 19.8 = 53.7 lbs**  
**(6.9 + 8.5 + 9.0 = 24.4 kg)**

### Final Answer:
The combined weight of Ash's first three Pokémon is **53.7 lbs (24.4 kg)**.

**Sources:**
- [List of Pokémon by height and weight - Pokémon DB](https://pokemondb.net/pokedex/stats/height-weight)
- [Bulbasaur Dimensions & Drawings - Dimensions.com](https://www.dimensions.com/element/bulbasaur)


In [52]:
@weave.op
@function_tool # <- we can use the decorator to automatically generate the tool schema
def exa_search_and_refine(query: str, num_results: int = 5) -> list[dict[str, str]]:
    """Perform a search query on the web and retrieve the most relevant URLs and web content.

    This function uses the Exa search API to find relevant web pages based on the query
    and returns their titles, text content, and URLs.

    Args:
        query: The search query. Use detailed, specific queries for better results.
               The quality of results depends on the specificity of the query.
        num_results: The number of search results to retrieve. Defaults to 5.

    Returns:
        A list of dictionaries, each containing:
            - title: The title of the web page
            - text: The text content of the web page
            - url: The URL of the web page
    """
    search_results = exa_client.search_and_contents(query=query, type='auto', num_results=num_results)

    @weave.op
    def refine_search_result(result, query):
        messages = [
            {"role":"system", "content": f"Your task is to extract from the search results only the info that is relevant to answer the query"},
            {"role": "user", "content": f"- query: {query}\n- Search result: {result}"}
        ]
        refined_search = call_model(model_name=MODEL_SMALL, messages=messages)
        return refined_search.content

    output = []
    for item, result in enumerate(search_results.results):
        console.print(f"Refining result {item+1}")
        refined_text = refine_search_result(result.text, query)
        output.append(
            {"title": result.title,
            "text": refined_text,
            "url": result.url
            }
        )
    return output

In [53]:
agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?", max_turns=3)
print(f"Final response: {state.final_assistant_content}")

weave: 🍩 https://wandb.ai/team-jup/london-workshop-2025/r/call/019a4eda-b174-7849-bee1-dd98dc1b6146


─────────────────────────────────────────────── Agent Loop Turn 1/3 ───────────────────────────────────────────────

Output()

Refining result 1

Refining result 2

...

─────────────────────────────────────────────── Agent Loop Turn 2/3 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 6, "b": 6}                                                                                          │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────── Agent Loop Turn 3/3 ───────────────────────────────────────────────

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ 🔧 Tool Call: add_numbers                                                                                       │
│                                                                                                                 │
│ Args: {"a": 12, "b": 6.9}                                                                                       │
│                                                                                                                 │
│ ✓ add_numbers executed successfully                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final response: None


Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.

## Evaluating our SimpleAgent

In [54]:
# Run this in colab

# !git clone https://github.com/morganmcg1/deep-research-bot/
# !mv deep-research-bot/data /data

In [55]:
import sys
from functools import partial
from pathlib import Path

# Add project root to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [57]:
from deep_research_bot.evaluation.eval import run_evaluation
from deep_research_bot.evaluation.eval_config import EvalConfig

MAX_TURNS=5

agent = SimpleAgent(
    model_name=MODEL_SMALL,
    system_message="You are an agent that has access to an advanced search engine. Please provide the user with the information they are looking for by using the search tool provided. Make sure to keep the sources. Always use tools to obtain reliable results. Return the final answer in markdown format.",
    tools=[exa_search_and_refine, add_numbers],
)

eval_config = EvalConfig(
    evaluation_name=f"SimpleAgent_max-turns-{MAX_TURNS}_{agent.model_name.split('/')[-1]}",
    trials=2,
    limit=20,
    judge_model="deepseek-ai/DeepSeek-R1-0528",
    weave_parallelism=4,
    queries=project_root / "data/prompt_data/query.jsonl",
    reference=project_root / "data/test_data/cleaned_data/reference.jsonl",
    criteria=project_root / "data/criteria_data/criteria.jsonl",
)

results = await run_evaluation(
    eval_config=eval_config,
    agent_callable=partial(agent.run, max_turns=MAX_TURNS),  # <- partial to limit the number of agent turns
)
results

ValueError: OPENAI_API_KEY is not set, please set it in the .env file.

In [59]:
!pip install nbconvert

In [61]:
!jupyter nbconvert --to html '/Users/admin/Downloads/01_simple_agent.ipynb'


[NbConvertApp] WARNING | pattern '/Users/admin/Downloads/01_simple_agent.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.ans